# Step1 Data_engineering 
The climate data for Hawaii is provided through two CSV files. Start by using Python and Pandas to inspect the content of these files and clean the data.

Create a Jupyter Notebook file called data_engineering.ipynb and use this to complete all of your Data Engineering tasks.

Use Pandas to read in the measurement and station CSV files as DataFrames.

Inspect the data for NaNs and missing values. You must decide what to do with this data.

Save your cleaned CSV files with the prefix clean_.

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
 
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pymysql
pymysql.install_as_MySQLdb()

 

## Reading in both the data files 'hawaii_measurements.csv' and 'hawaii_stations.csv'

In [2]:
measurements_df = pd.read_csv('Resources/hawaii_measurements.csv')
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [3]:
len(measurements_df) # lenght 19550

19550

In [4]:
measurements_df['station'].value_counts()

USC00519281    2772
USC00519397    2724
USC00513117    2709
USC00519523    2669
USC00516128    2612
USC00514830    2202
USC00511918    1979
USC00517948    1372
USC00518838     511
Name: station, dtype: int64

In [5]:
#measurements_df.groupby(['station','prcp']).size()

In [6]:
measurements_df.dtypes


station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [7]:
# Inspect the data for Nan and missing data
measurements_df.isnull() # check to see if any data is Nan in the series
measurements_df.isnull().values.any()# just check overall to see if any missing  
measurements_df.isnull().sum().sum() # total number of all the missing values
measurements_df.isnull().sum() # all the columns which have missing data

station       0
date          0
prcp       1447
tobs          0
dtype: int64

### Cleaning up the data by dropping all NaN's in the dataframe('hawaii_measurements.csv')

In [8]:
# fill missing data Nan with zero 
#fillna_measurements_df = measurements_df.fillna(0)
#fillna_measurements_df.head()

In [9]:
# drop all the NaN data
fillna_measurements_df = measurements_df.dropna(how = 'any')
fillna_measurements_df.count()

 

station    18103
date       18103
prcp       18103
tobs       18103
dtype: int64

In [10]:
fillna_measurements_df.to_csv('Resources/clean_hawaii_measurements.csv', index=True)

In [11]:
#cat 'Resources/clean_hawaii_measurements.csv'

In [12]:
pd.Series(list(fillna_measurements_df.index))

0            0
1            1
2            2
3            3
4            5
5            6
6            7
7            8
8            9
9           10
10          11
11          12
12          13
13          14
14          15
15          16
16          17
17          18
18          19
19          20
20          21
21          22
22          23
23          24
24          25
25          27
26          28
27          30
28          31
29          32
         ...  
18073    19513
18074    19514
18075    19515
18076    19516
18077    19517
18078    19518
18079    19519
18080    19520
18081    19521
18082    19522
18083    19523
18084    19524
18085    19525
18086    19526
18087    19527
18088    19529
18089    19530
18090    19533
18091    19534
18092    19535
18093    19536
18094    19538
18095    19540
18096    19541
18097    19542
18098    19543
18099    19545
18100    19547
18101    19548
18102    19549
Length: 18103, dtype: int64

In [13]:
fillna_measurements_df.index

Int64Index([    0,     1,     2,     3,     5,     6,     7,     8,     9,
               10,
            ...
            19536, 19538, 19540, 19541, 19542, 19543, 19545, 19547, 19548,
            19549],
           dtype='int64', length=18103)

In [14]:
fillna_measurements_df.to_csv('Resources/clean_hawaii_measurements.csv', index=True)
new_measurements_df = pd.read_csv('Resources/clean_hawaii_measurements.csv', index_col=0)
new_measurements_df['id'] = pd.Series(list(fillna_measurements_df.index))
new_measurements_df.head()

,station,date,prcp,tobs,id
0,USC00519397,2010-01-01,0.08,65,0.0
1,USC00519397,2010-01-02,0.00,63,1.0
2,USC00519397,2010-01-03,0.00,74,2.0
3,USC00519397,2010-01-04,0.00,76,3.0
5,USC00519397,2010-01-07,0.06,70,6.0


### No need to cleanup any data as there are no missing nor Nan's in the dataframe ('hawaii_stations.csv')

In [15]:
pd.Series(list(new_measurements_df.index))

0            0
1            1
2            2
3            3
4            5
5            6
6            7
7            8
8            9
9           10
10          11
11          12
12          13
13          14
14          15
15          16
16          17
17          18
18          19
19          20
20          21
21          22
22          23
23          24
24          25
25          27
26          28
27          30
28          31
29          32
         ...  
18073    19513
18074    19514
18075    19515
18076    19516
18077    19517
18078    19518
18079    19519
18080    19520
18081    19521
18082    19522
18083    19523
18084    19524
18085    19525
18086    19526
18087    19527
18088    19529
18089    19530
18090    19533
18091    19534
18092    19535
18093    19536
18094    19538
18095    19540
18096    19541
18097    19542
18098    19543
18099    19545
18100    19547
18101    19548
18102    19549
Length: 18103, dtype: int64

In [16]:
stations_df = pd.read_csv('Resources/hawaii_stations.csv')
stations_df.to_csv('Resources/clean_hawaii_stations.csv', index=True)
new_stations_df = pd.read_csv('Resources/clean_hawaii_stations.csv' ,index_col=0)
new_stations_df['id'] = pd.Series(list(stations_df.index))
new_stations_df.head()

,station,name,latitude,longitude,elevation,id
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0,0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6,1
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0,2
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9,3
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6,4


In [17]:
l = []
for row in new_measurements_df.iterrows():
    l.append(row)
    
l[0][1]

station    USC00519397
date        2010-01-01
prcp              0.08
tobs                65
id                   0
Name: 0, dtype: object

In [18]:
l = []
for row in new_stations_df.iterrows():
    l.append(row)
    
l[0][1]

station               USC00519397
name         WAIKIKI 717.2, HI US
latitude                  21.2716
longitude                -157.817
elevation                       3
id                              0
Name: 0, dtype: object

# Step 2 - Database Engineering
 
Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.

Use Pandas to read your cleaned measurements and stations CSV data.

Use the engine and connection string to create a database called hawaii.sqlite.

Use declarative_base and create ORM classes for each table.

You will need a class for Measurement and for Station.
Make sure to define your primary keys.
Once you have your ORM classes defined, create the tables in the database using create_all.

In [19]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float
 


In [20]:
#Delete the sqlite db before rerunning my code all over again!
#!rm hawaii.sqlite
!rm dropna_hawaii.sqlite

In [21]:
# Define a Measurements class and/or model of Measurement
 
class Measurement(Base):
    __tablename__ = "measurements" 
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String) 
    prcp = Column(Float)  
    tobs = Column(Integer) 
    
 

In [22]:
# Define a Stations class and/or model of Stations
 
class Station(Base):
    __tablename__ = "stations" 
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String(255)) 
    latitude = Column(Float)  
    longitude = Column(Float) 
    elevation = Column(Float)
 

In [23]:
# Create Database Connection
engine = create_engine("sqlite:///dropna_hawaii.sqlite")

# fillna(0) decided to compare both the data results, with dropping, and filling 0.
#engine = create_engine("sqlite:///hawaii.sqlite")

In [24]:
Base.metadata.create_all(engine)

In [25]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [26]:
session.commit()

In [27]:
#Write records stored in a DataFrame to a SQL database.
#with engine.connect() as conn,conn.begin():
new_measurements_df.to_sql(con=engine, index_label='id', name='measurements',if_exists='append',index=False) 
new_stations_df.to_sql(con=engine, index_label='id', name='stations',if_exists='append',index=False)


In [28]:
list(map(lambda x: x.id, session.query(Measurement).all()))

[0,
 1,
 2,
 3,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 27,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 44,
 45,
 46,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 62,
 63,
 64,
 65,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 125,
 126,
 127,
 128,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198

In [29]:
list(map(lambda x: x.id, session.query(Station).all()))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [30]:
session.query(Measurement).all()

 ...]

In [31]:
session.query(Station).all()

In [32]:
session.query(Measurement.date, Measurement.prcp)

In [33]:
#Saving data from sqlalchemy to pandas dataframe

with engine.connect() as conn,conn.begin():
    data1=pd.read_sql_table('measurements',conn)
    data2=pd.read_sql_table('stations',conn)
    
     

In [34]:
data1.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,6,USC00519397,2010-01-07,0.06,70


In [35]:
data2.head()

,id,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [39]:
# To inspect and see if my database has the tables 
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection

engine = create_engine("sqlite:///dropna_hawaii.sqlite")
insp = reflection.Inspector.from_engine(engine)
print(insp.get_table_names())

['measurements', 'stations']


In [40]:
session.query(Measurement.date, Measurement.prcp).filter(Measurement.date.between('2016-08-23', '2017-08-23'))

In [41]:
Measurement.date.between('2016-08-23', '2017-08-23')